<a href="https://colab.research.google.com/github/jespimentel/rag_simplificado/blob/main/rag_simplificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exemplo de RAG
### com Chroma DB e LiteLLM



Baseado nos artigos:
* https://www.nb-data.com/p/simple-rag-implementation-with-contextual
* https://medium.com/@rahultiwari065/unlocking-the-power-of-sentence-embeddings-with-all-minilm-l6-v2-7d6589a5f0aa

---

In [ ]:
!pip install -q chromadb pypdf2 sentence-transformers litellm langchain
!pip install -q sympy

In [2]:
import os
import PyPDF2
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import litellm
from litellm import completion
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import Markdown, display
from google.colab import userdata

In [18]:
model = 'deepseek/deepseek-chat'
api_key = userdata.get('DEEPSEEK_KEY')

os.environ['LITELLM_LOG'] = 'DEBUG'

# HUGGINGFACE_TOKEN = userdata.get('hugging_face_read')
HF_TOKEN = userdata.get('hugging_face_read')

In [4]:
def extract_text_from_pdfs(folder_path):
    all_text = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    all_text += page.extract_text()
    return all_text

pdf_folder = "/content/drive/MyDrive/pdfs_para_rag"
all_text = extract_text_from_pdfs(pdf_folder)

In [5]:
print(len(all_text))

3359036


In [6]:
all_text[6000:7000]

'......... 117 7 - Tema: Estudo do MPPE - A efetividade na tutela dos direitos humanos das vítimas ...... 124 8 - Tema: Estudo do MPPR - Planejamento investigatório .................................................... 1359 - Tema: Estudo do MPRJ - A Atuação Coletiva Especializada na Segurança Pública a partir da Resolução GPGJ nº 2.401/2021 – uma breve exposição dos casos ....................................... 143 10 - Tema: Estudo do MPRO - Estudo jurídico sobre as controvérsias atuais nas Cortes  Superiores \nreferentes ao ingresso em domicílio sem mandado judicial .................................................. 15011 - Tema: Estudo do MPRS - Cadeia De Custódia De Whatsapp: breve considerações sobre  decisões do Superior Tribunal de Justiça ............................................................................... 158 12 - Tema: Estudo do MPSC - Destinação de verbas oriundas de prestação pecuniária  decorrente \nde transação penal, suspensão condicional do processo e acordo 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Size of each chunk
    chunk_overlap=100,  # Overlap between chunks to maintain context
    separators=["\n\n", "\n", " ", ""]  # Splitting hierarchy
)

chunks = text_splitter.split_text(all_text)

In [8]:
print(len(chunks))

3830


In [9]:
print(chunks[2000])

fato de haver denúncia contra parte dos investigados pelo crime de organização criminosa (processo 
n. 1541165 -65.2021.8.26.0050) não faz com que necessariamente haja conexão entre os feitos, 
mesmo porque ambos estão em fases notor iamente distintas da marcha processual. (ﬂs. 164/168)  
Vieram os autos a esta Cheﬁa Institucional para solução do impasse (ﬂs. 170).  
Eis a síntese do necessário.  
A remessa se fundamenta no art. 115 da Lei Complementar Estadual n.º 734/93, encontrando -
se conﬁgurado, portanto, o incidente supramencionado entre promotores de justiça.   
C o m o  d e s t a c a  H U G O  N I G R O  M A Z Z I L L I ,  t a l  i n c i d e n t e  t e m  l u g a r  q u a n d o  o  m e m b r o  d o  
Ministério Público nega a própria atribuição funcional e a atribui a outro, que já a tenha recusado (conﬂito negativo), ou quando dois ou mais deles manifestam, simultaneamente, atos que importem


In [13]:
# Initialize a persistent ChromaDB client
client = chromadb.PersistentClient(path="/content/drive/MyDrive/pdfs_para_rag/chroma_db")

# Load the SentenceTransformer model for text embeddings
text_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Delete existing collection (if needed)
try:
    client.delete_collection(name="knowledge_base")
    print("Deleted existing collection: knowledge_base")
except Exception as e:
    print(f"Collection does not exist or could not be deleted: {e}")

# Create a new collection for text embeddings
collection = client.create_collection(name="knowledge_base")

# Add text chunks to the collection
for i, chunk in enumerate(chunks):
    # Generate embeddings for the chunk
    embedding = text_embedding_model.encode(chunk)

    # Add to the collection with metadata
    collection.add(
        ids=[f"chunk_{i}"],  # Unique ID for each chunk
        embeddings=[embedding.tolist()],  # Embedding vector
        metadatas=[{"source": "pdf", "chunk_id": i}],  # Metadata
        documents=[chunk]  # Original text
    )

Deleted existing collection: knowledge_base


In [14]:
def semantic_search(query, top_k=4):
    # Generate embedding for the query
    query_embedding = text_embedding_model.encode(query)

    # Query the collection
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
    )
    return results

In [36]:
# Example query
query = "o que é violência de gênero?"
results = semantic_search(query)

# Display results
for i, result in enumerate(results['documents'][0]):
    print(f"Result {i+1}:\n{result}\n")
    print("*" * 60 + '\n')

Result 1:
cumprimento da pena em regime aberto.  
  
    
  
 
    
 
   
 
   
 
   
 
   
 
   
 
   
Núcleo de Gênero    
1661 
 
  
Boletim Criminal COMENTADO  
– 
  
n° 2  
44  
- 
Setembro  
- 
202  
3 
  
1.  Tema: Barreiras para acesso à Justiça  
 
  
Quando se fala em violência de gênero, existem obstáculos ao direito à igualdade no acesso à 
justiça, que podem ser históricos, como a percepção da violência doméstica como um problema 
privado, a discriminação de gênero (viés de gênero ou gender bias ) e discriminações previstas 
pelo Estado, tais como, onde ainda existe, a aceitação legal de estupro marital. Podem, 
também, ser  estruturais, dos quais são exemplos a minimização da violência contra a mulher, a culpabilização da vítima e a descrença indevida na  palavra dela. Podem, da mesma forma, ser

************************************************************

Result 2:
“as declarações da vítima qualiﬁcam -se como meio de prova, de 
inquestionável importância quando se discu

In [37]:
# Set up LiteLLM

def generate_response(query, context):
    # Combine the query and context for the prompt
    prompt = f"Query: {query}\nContext: {context}\nAnswer:"

    # Call the model via LiteLLM
    response = completion(
        model=model,
        messages=[{"content": prompt, "role": "user"}],
        api_key= api_key
    )

    # Extract and return the generated text
    return response['choices'][0]['message']['content']

# Retrieve the top results from semantic search
search_results = semantic_search(query)
context = "\n".join(search_results['documents'][0])

# Generate a response using the retrieved context
response = generate_response(query, context)

print("Generated Response:\n")
display(Markdown(response))

Generated Response:



**Violência de gênero** é qualquer forma de violência direcionada a uma pessoa em razão de seu gênero, identidade de gênero ou expressão de gênero, sustentada por desigualdades estruturais e relações de poder historicamente desiguais entre homens e mulheres, bem como por normas sociais que reforçam estereótipos e discriminações.  

### **Características e Contexto (com base no texto fornecido):**  
1. **Raiz Estrutural**:  
   - Decorrente de desigualdades históricas, culturais e materiais (ex.: dependência financeira, cultura do estupro, erotização da subordinação).  
   - Inclui violências "corretivas", que buscam impor papéis de gênero tradicionais (ex.: violência contra pessoas LGBTQIA+ que desafiam o binarismo).  

2. **Manifestações**:  
   - **Violência doméstica**: Tradicionalmente tratada como "problema privado", com vítimas frequentemente culpabilizadas.  
   - **Barreiras no acesso à justiça**: Descrença na palavra da vítima (especialmente em crimes sexuais), minimização da violência e exigência de provas adicionais além do relato da vítima.  
   - **Discriminação institucional**: Ex.: leis que historicamente permitiam estupro marital ou negavam proteção a grupos marginalizados (como homens trans).  

3. **Enquadramento Legal**:  
   - A **Lei Maria da Penha (Lei 11.340/2006)** reconhece a violência de gênero contra mulheres e prevê políticas públicas para combatê-la, incluindo educação sobre equidade de gênero.  
   - **Protocolos de julgamento com perspectiva de gênero** destacam a necessidade de valorizar o relato da vítima, considerando sua vulnerabilidade processual (ex.: hipossuficiência em casos de violência sexual).  

4. **Aspectos Sociais**:  
   - Envolve estereótipos que naturalizam a dominação masculina e a submissão feminina.  
   - Afeta não apenas mulheres cisgênero, mas também pessoas trans e não binárias, cujas identidades são frequentemente alvo de violência "corretiva".  

### **Exemplos citados no texto**:  
- **Estupro marital**: Historicamente tolerado por leis que negavam autonomia feminina.  
- **Culpabilização da vítima**: Exigir que a vítima prove que "não consentiu" com a violência.  
- **Violência contra homens trans**: Corpos lidos como "desviantes" do binarismo de gênero.  

### **Conclusão**:  
A violência de gênero é um fenômeno complexo, alimentado por estruturas sociais e jurídicas que perpetuam desigualdades. Seu combate exige mudanças culturais, educacionais e legais, como as previstas na Lei Maria da Penha e em protocolos que priorizam a escuta qualificada das vítimas.  

*(Resposta sintetizada a partir do contexto jurídico e social apresentado no boletim.)*

In [38]:
search_results

{'ids': [['chunk_3357', 'chunk_3650', 'chunk_3592', 'chunk_1603']],
 'embeddings': None,
 'documents': [['cumprimento da pena em regime aberto.  \n  \n    \n  \n \n    \n \n   \n \n   \n \n   \n \n   \n \n   \n \n   \nNúcleo de Gênero    \n1661 \n \n  \nBoletim Criminal COMENTADO  \n– \n  \nn° 2  \n44  \n- \nSetembro  \n- \n202  \n3 \n  \n1.  Tema: Barreiras para acesso à Justiça  \n \n  \nQuando se fala em violência de gênero, existem obstáculos ao direito à igualdade no acesso à \njustiça, que podem ser históricos, como a percepção da violência doméstica como um problema \nprivado, a discriminação de gênero (viés de gênero ou gender bias ) e discriminações previstas \npelo Estado, tais como, onde ainda existe, a aceitação legal de estupro marital. Podem, \ntambém, ser  estruturais, dos quais são exemplos a minimização da violência contra a mulher, a culpabilização da vítima e a descrença indevida na  palavra dela. Podem, da mesma forma, ser',
   '“as declarações da vítima qualiﬁcam -